In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [2]:
df = pd.read_sql("""
SELECT patient_id, sex, birth_date, created_at
FROM patient
ORDER BY patient_id;
""", engine)

df


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "patient" does not exist
LINE 3: FROM patient
             ^

[SQL: 
SELECT patient_id, sex, birth_date, created_at
FROM patient
ORDER BY patient_id;
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [5]:
pd.read_sql("SELECT sex, COUNT(*) n FROM patient GROUP BY sex ORDER BY n DESC;", engine)


,sex,n
0,O,1
1,M,1
2,F,1
